In [1]:
# import
import os, sys
import numpy as np

# import plotting libraries
import matplotlib.pyplot as plt
# import nctpy functions
sys.path.extend(['/Users/lindenmp/Google-Drive-Personal/work/research_projects/nctpy/src'])
from nctpy.energies import integrate_u, get_control_inputs
from nctpy.utils import matrix_normalization, convert_states_str2int, normalize_state
from nctpy.plotting import set_plotting_params
set_plotting_params()

# Using get_control_inputs

This notebook covers examples of different ways in which get_control_inputs can used to derive control energy

### Load adjacency

In [2]:
# directory where data is stored
datadir = "/Users/lindenmp/Google-Drive-Personal/work/research_projects/nctpy/data"
resultsdir = "/Users/lindenmp/Google-Drive-Personal/work/research_projects/nctpy/results"
adjacency_file = "pnc_schaefer200_Am.npy"

# load adjacency matrix
adjacency = np.load(os.path.join(datadir, adjacency_file))
n_nodes = adjacency.shape[0]
print(adjacency.shape)

(200, 200)


### Get initial and target state

In [3]:
# load node-to-system mapping
system_labels = list(
    np.loadtxt(os.path.join(datadir, "pnc_schaefer200_system_labels.txt"), dtype=str)
)

# use list of system names to create states
states, state_labels = convert_states_str2int(system_labels)

# extract initial state
initial_state = states == state_labels.index("Vis")

# extract target state
target_state = states == state_labels.index("Default")

# normalize state magnitude
initial_state = normalize_state(initial_state)
target_state = normalize_state(target_state)

# Get energy

By default, we compute $x$ and $u$ using the following settings:
1. Time system: continuous
2. Control set: uniform full
3. Time horizon: 1
4. The constraints on $x$ are equal to the constraints on $u$ (*rho*)
5. The constraints on $x$ are applied to all system nodes (*trajectory_constraints*)
6. Amplitude of $x$ and $u$ are constrained toward zero (*reference_state*)

You can compute $x$ and $u$ using the above settings like so:

In [4]:
system = "continuous"  # choose time system
adjacency_norm = matrix_normalization(adjacency, system=system, c=1)  # normalize adjacency
control_set = np.eye(n_nodes)  # uniform full control set
time_horizon = 1  # time horizon
rho = 1  # x and u constrained equally
trajectory_constraints = np.eye(n_nodes)  # x is constrained over all nodes
reference_state = np.zeros(n_nodes)  # x and u constrained toward zero activity

# get the state trajectory and the control inputs
state_trajectory, control_signals, numerical_error = get_control_inputs(
    A_norm=adjacency_norm,
    T=time_horizon,
    B=control_set,
    x0=initial_state,
    xf=target_state,
    system=system,
    rho=rho,
    S=trajectory_constraints,
    xr=reference_state,
)
node_energy = integrate_u(control_signals)
energy = np.sum(node_energy)

print("energy: {0:.2f}".format(energy))

energy: 2604.71


## Variations

All of the below code cells represent variations to the above default settings. Changes are noted in comments.

### You want to vary the constraints on $x$?

Note, constraints on $u$ cannot be change; the control signals are always minimized in the cost function.
The only thing that can be varied is the extent to which $x$ is constrained alongside $u$.
Reducing $rho$ below 1 increases the extent to which $x$ adds to the cost function alongside $u$.
Conversely, increasing $rho$ beyond 1 reduces the contribution of $x$, thus increasing the relative prioritization of $u$.

In [5]:
system = "continuous"  # choose time system
adjacency_norm = matrix_normalization(adjacency, system=system, c=1)  # normalize adjacency
control_set = np.eye(n_nodes)  # uniform full control set
time_horizon = 1  # time horizon
rho = 0.5  # x constrained more than u  <-- NOTE CHANGE.
trajectory_constraints = np.eye(n_nodes)  # x is constrained over all nodes
reference_state = np.zeros(n_nodes)  # x and u constrained toward zero activity

# get the state trajectory and the control inputs
state_trajectory, control_signals, numerical_error = get_control_inputs(
    A_norm=adjacency_norm,
    T=time_horizon,
    B=control_set,
    x0=initial_state,
    xf=target_state,
    system=system,
    rho=rho,
    S=trajectory_constraints,
    xr=reference_state,
)
node_energy = integrate_u(control_signals)
energy = np.sum(node_energy)

print("energy: {0:.2f}".format(energy))

energy: 2686.91


In [6]:
system = "continuous"  # choose time system
adjacency_norm = matrix_normalization(adjacency, system=system, c=1)  # normalize adjacency
control_set = np.eye(n_nodes)  # uniform full control set
time_horizon = 1  # time horizon
rho = 1.5  # x constrained less than u  <-- NOTE CHANGE.
trajectory_constraints = np.eye(n_nodes)  # x is constrained over all nodes
reference_state = np.zeros(n_nodes)  # x and u constrained toward zero activity

# get the state trajectory and the control inputs
state_trajectory, control_signals, numerical_error = get_control_inputs(
    A_norm=adjacency_norm,
    T=time_horizon,
    B=control_set,
    x0=initial_state,
    xf=target_state,
    system=system,
    rho=rho,
    S=trajectory_constraints,
    xr=reference_state,
)
node_energy = integrate_u(control_signals)
energy = np.sum(node_energy)

print("energy: {0:.2f}".format(energy))

energy: 2586.67


### You want to only constrain u and not x?

Note, this is what we refer to as *minimum control energy*; the energy where only the control signals are constrained

In [7]:
system = "continuous"  # choose time system
adjacency_norm = matrix_normalization(adjacency, system=system, c=1)  # normalize adjacency
control_set = np.eye(n_nodes)  # uniform full control set
time_horizon = 1  # time horizon
#  <-- NOTE REMOVAL OF RHO
trajectory_constraints = np.zeros((n_nodes, n_nodes))  # x is *not* constrained  <-- NOTE CHANGE
reference_state = np.zeros(n_nodes)  # u constrained toward zero activity

# get the state trajectory and the control inputs
state_trajectory, control_signals, numerical_error = get_control_inputs(
    A_norm=adjacency_norm,
    T=time_horizon,
    B=control_set,
    x0=initial_state,
    xf=target_state,
    system=system,
    # <-- NOTE REMOVAL OF RHO
    S=trajectory_constraints,
    xr=reference_state,
)
node_energy = integrate_u(control_signals)
energy = np.sum(node_energy)

print("energy: {0:.2f}".format(energy))

energy: 2570.65


### You want to constrain $x$ and $u$ toward the target state instead of toward zero?

In [8]:
system = "continuous"  # choose time system
adjacency_norm = matrix_normalization(adjacency, system=system, c=1)  # normalize adjacency
control_set = np.eye(n_nodes)  # uniform full control set
time_horizon = 1  # time horizon
rho = 1  # x and u constrained equally
trajectory_constraints = np.eye(n_nodes)  # x is constrained over all nodes
reference_state = target_state # x and u constrained toward target start  <-- NOTE CHANGE

# get the state trajectory and the control inputs
state_trajectory, control_signals, numerical_error = get_control_inputs(
    A_norm=adjacency_norm,
    T=time_horizon,
    B=control_set,
    x0=initial_state,
    xf=target_state,
    system=system,
    rho=rho,
    S=trajectory_constraints,
    xr=reference_state,
)
node_energy = integrate_u(control_signals)
energy = np.sum(node_energy)

print("energy: {0:.2f}".format(energy))

energy: 2607.39


### You want to use a discrete time system instead?

Note, time horizon must be >1 for discrete time systems. In a discrete system, time is broken into
discrete steps of 1, which means time_horizon=1 would be only 1 time step.

In [9]:
system = "discrete"  # choose time system  <-- NOTE CHANGE
adjacency_norm = matrix_normalization(adjacency, system=system, c=1)  # normalize adjacency
control_set = np.eye(n_nodes)  # uniform full control set
time_horizon = 10  # time horizon  <-- NOTE CHANGE. TIME HORIZON MUST BE >1 FOR DISCRETE TIME SYSTEMS
rho = 1  # x and u constrained equally
trajectory_constraints = np.eye(n_nodes)  # x is constrained over all nodes
reference_state = np.zeros(n_nodes)  # x and u constrained toward zero activity

# get the state trajectory and the control inputs
state_trajectory, control_signals, numerical_error = get_control_inputs(
    A_norm=adjacency_norm,
    T=time_horizon,
    B=control_set,
    x0=initial_state,
    xf=target_state,
    system=system,
    rho=rho,
    S=trajectory_constraints,
    xr=reference_state,
)
node_energy = integrate_u(control_signals)
energy = np.sum(node_energy)

print("energy: {0:.2f}".format(energy))

energy: 0.51
